In [2]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from modules import solver, presolver
from pyballistics import ozvb_lagrange, get_powder_names
from tqdm import tqdm

## Нахождение начальных точек
формируется таблица с начальными точками для каждого пороха

In [3]:
tabel_init_points = pd.read_csv("data/init_points.csv")
tabel_init_points.loc[tabel_init_points['success']==False]

,dbname,wq,ro,reason,success


## Решение обратной задачи

In [4]:
tabel_result = list()

In [5]:
%%time

for _, row in tqdm(tabel_init_points.iterrows()):
    initial_dict = presolver.init_dict(row['dbname'], row['wq'], row['ro'])
    #print("name: ", row['dbname'], " wq: ", row['wq'], " ro ", row['ro'])
    solv = solver.Solver(initial_dict)
    solv.solution()
    tabel_result.append(solv.get_solution())
        

0it [00:00, ?it/s]

Optimization terminated successfully.
         Current function value: 46377.918725
         Iterations: 105
         Function evaluations: 200
Optimization terminated successfully.
         Current function value: 46377.918725
         Iterations: 93
         Function evaluations: 174
No success in result
Optimization terminated successfully.
         Current function value: 992.757071
         Iterations: 87
         Function evaluations: 168
Optimization terminated successfully.
         Current function value: 928.111222
         Iterations: 108
         Function evaluations: 206
Optimization terminated successfully.
         Current function value: 8367.414202
         Iterations: 74
         Function evaluations: 147
Optimization terminated successfully.
         Current function value: 859.845700
         Iterations: 106
         Function evaluations: 204
Optimization terminated successfully.
         Current function value: 1264.952612
         Iterations: 121
         Function

E:\PROGRAMMING PROJECT\Jupyter project\BMSTU\5th year\Gun Projecting\modules\solver.py:223: RuntimeWarning: divide by zero encountered in true_divide
  self.n_real = self.pressure_tube/self.pressure


Optimization terminated successfully.
         Current function value: 19869.114167
         Iterations: 65
         Function evaluations: 132
Optimization terminated successfully.
         Current function value: 855.711869
         Iterations: 103
         Function evaluations: 191
Optimization terminated successfully.
         Current function value: 723.142407
         Iterations: 70
         Function evaluations: 145
Optimization terminated successfully.
         Current function value: 652.738360
         Iterations: 73
         Function evaluations: 137
Optimization terminated successfully.
         Current function value: 747.325416
         Iterations: 86
         Function evaluations: 159
Optimization terminated successfully.
         Current function value: 759.903356
         Iterations: 84
         Function evaluations: 162
Optimization terminated successfully.
         Current function value: 679.419270
         Iterations: 150
         Function evaluations: 308
Optimizat

Optimization terminated successfully.
         Current function value: 521.089759
         Iterations: 86
         Function evaluations: 163
No success in result
Optimization terminated successfully.
         Current function value: 558.145422
         Iterations: 74
         Function evaluations: 149
Optimization terminated successfully.
         Current function value: 7124.136911
         Iterations: 74
         Function evaluations: 149
Optimization terminated successfully.
         Current function value: 575.132041
         Iterations: 125
         Function evaluations: 240
Optimization terminated successfully.
         Current function value: 589.942197
         Iterations: 76
         Function evaluations: 156
Optimization terminated successfully.
         Current function value: 135592.532142
         Iterations: 110
         Function evaluations: 269
Optimization terminated successfully.
         Current function value: 595.375376
         Iterations: 85
         Function eva

In [6]:
result = copy.deepcopy(tabel_result)

In [29]:
table = list()
for row in result:
    if row == {None}:
        continue
    table.append(row)

In [30]:
table = pd.DataFrame(table)

In [31]:
#table.to_csv("data/table_result.csv", index=False)

## Обработка

In [32]:
table.head()

,name,succes,wq,ro,mass
0,ДРП,True,3.869251,1011.404983,2650.941081
1,КЗДП,True,3.869251,1011.404979,2650.945574
2,СФ 033,True,1.935400,306.411056,992.756855
3,ВТ,True,1.613720,341.222446,928.111264
4,ВТМ,True,1.044243,429.070729,1769.155477


In [38]:
table.sort_values(by=['mass'])

,name,succes,wq,ro,mass
81,УГ-2,True,0.866869,89.828520,300.690783
34,12/7 В/А,True,1.327616,885.092813,301.055455
36,12/7,True,1.547683,897.894231,305.972314
41,15/1 тр В/А,True,1.383710,866.740867,306.791717
40,ДГ-4 13/1,True,1.494576,785.134073,315.032846
...,...,...,...,...,...
0,ДРП,True,3.869251,1011.404983,2650.941081
1,КЗДП,True,3.869251,1011.404979,2650.945574
17,ВТХ-20,True,2.926365,498.787279,3182.752509
57,УГ-1,True,8.138917,1132.513282,3280.681214
